In [ ]:
from uuid import uuid4
import os
import pandas as pd
from databricks.sdk import WorkspaceClient

from ag_vision.drone.ingest import DroneDataIngest

os.environ.pop('DATABRICKS_HOST', None)
os.environ.pop('DATABRICKS_TOKEN', None)

w = WorkspaceClient(profile='tnau')

In [ ]:
w.config.host

In [ ]:
# Change the following to the location where the local data is stored.
parent_folder_path = '/Users/danielwilliams/Documents/Field Data/drone_upload_test'
raw_image_dir = f'{parent_folder_path}/raw_flight_data/'
files = os.listdir(raw_image_dir)
files = [raw_image_dir + x for x in files]
files

In [ ]:
flight_metadata = {
    "id": str(uuid4()),
    "name": "corn_health_assessment_flight",  # this will be the mission name in the folder path
    "task": "drone_phenotyping",
    "location": {
        "site": "TNAU-Coimbatore".lower(),  # Try to keep these standard ORG-Site EX CIAT-Cali, or CIAT-Arusha....
        "field": "West_Field_03".lower(),
        # what is the name of the field that the trial was run on? The field and location can be the same name if there is no difference.
        "location": "Section_B".lower()
        # The location corresponds to a specific field book. If data is in EBS match the location name.
    },
    "trialProperties": {
        "name": "BETS"  # What is the name of the trial, a trial usually has multiple locations.
    },
    "drone_acquisition_properties": {
        'date': '11/25/2024',
        'drone_make': "DJI",
        'drone_model': "Phantom Mavic 4 Enterprise",
        'camera_make': "Sony",
        'camera_model': "UCM-R",
        'groundControlPoints': True,
        'reflectancePanels': True,
        'reflectancePanelType': 'Micasense',
        'flightHeight': 90.0,
        'horizontalOverlapPercentage': 75.0,
        'verticalOverlapPercentage': 75.0,
        'gpsQuality': "RTK",
        'multispecChannels': ["Red", "Green", "Blue", "NIR"]
    },
    "agronomic_properties": {
        "crop_type": "corn",  # Required
        "growth_stage": "VT",  # optional
        "soil_color": None,
        "weed_pressure": None,
        "irrigation_level": None,
        "tillage_type": None,
        "fertilizer_level": None
    }
}

In [ ]:
# This points to the aps1-prod-tnau-fg workspace _YOU SHOULD NOT NEED TO CHANGE THIS.
workspace_bucket = '/Volumes/aps1_prod_tnau_fg_catalog_1336582592012881/tier1_raw/data'

ingest = DroneDataIngest(platform='local',  # DONT CHANGE THIS
                         cloud_bucket=workspace_bucket, # Set this to the bucket you want to save the data to.
                         cloud_client=w, # should not need to change.
                         flight_date='11/25/2024',
                         plot_boundary_key=f"{parent_folder_path}/plot_boundary.geojson",
                         gcp_key=f"{parent_folder_path}/gcp_details.geojson",
                         orthomosaic_key=f"{parent_folder_path}/30112024_ORTHO.tif",
                         dem_key=f"{parent_folder_path}/30112024_DSM.tif",
                         flight_metadata_key=f"{parent_folder_path}/flight_details.json")  # If this does not exist then this just tells the code were to save it so it can be uploaded.

In [ ]:
ingest.load_metadata_from_dict(metadata_dict=flight_metadata)

# season is needed to generate the mission dir, This has lots of Validation and will throw assert errors.
ingest.add_season_code_to_metadata(year=2025,
                                   country='IND',
                                   crop='maize',
                                   time_of_year='spring')

# This is the main dir where all the data will be stored.
ingest.generate_drone_mission_dir_path()

In [ ]:
# This saves the metadata locally do it can be uploaded.
ingest.save_flight_metadata_to_json_local()

In [ ]:
# This will be a list of all the raw files from the flight eg: nav, bin, tif, and jpg files.
ingest.generate_raw_ingest_df(file_list=files)

In [ ]:
def determine_camera_type(f_name):
    if "_MS_" in f_name:
        return 'multi-spec'
    elif ".JPG" in f_name:
        return 'rgb'
    else:
        return 'flight_data'

ingest.raw_ingest_df['camera'] = ingest.raw_ingest_df['file_name'].apply(determine_camera_type)

flight_data = ingest.raw_ingest_df[ingest.raw_ingest_df['file_type'] == 'flight_data']

f_ingest_df_list = []

# The flight data needs to be saved with both camera data sets.
for idx, df in ingest.raw_ingest_df.groupby('camera'):
    if idx in ['rgb', 'multi-spec']:
        out_df = pd.concat([df, flight_data])
        out_df['camera'] = idx
        f_ingest_df_list.append(out_df)

f_ingest_df = pd.concat(f_ingest_df_list)


In [ ]:
ingest.raw_ingest_df = f_ingest_df

In [ ]:
# generates the new path names where to save it in databricks
ingest.generate_raw_image_dst_path_name()

In [ ]:
ingest.raw_ingest_df.sample(10)

In [ ]:
# Upload the metadata to databricks
ingest.upload_metadata_to_db()

In [ ]:
# uploads the plot boundaries to databricks
ingest.upload_plot_boundary_to_db()

In [ ]:
# Uploads the ground control points to databricks
ingest.upload_gcp_to_db()

In [ ]:
# uploads all the raw flight data to databricks
ingest.upload_raw_flight_data_to_db()

In [ ]:
# If you have a dem generated you can upload it so we don't need to reprocess.
ingest.upload_dem_to_db(method='agisoft',  # the method used to generate the dem.
                        dem_date='11/25/2024',  # the date the dem was generated.
                        file_name='dem.tif')

In [22]:
# f"{drone_flight_path}/orthomosaic/{method}_{date}/{image_name}"
ingest.upload_orthomosaic_to_db(method='agisoft',  # the method used to generate the ortho.
                                ortho_date='11/25/2024', # the date the ortho was generated.
                                camera='multi-spec', # The camera that was used.
                                file_name='ms.tif')  # if RGB then you can name it rgb.tif

Uploading: 100%|██████████| 2.00G/2.00G [20:37<00:00, 1.74MB/s]
